In [1]:
import torch

import pandas as pd
import numpy as np

In [2]:
x = torch.rand(5,3)
print(x)
y = torch.ones(5,3)
print(y)

tensor([[0.7739, 0.9195, 0.8252],
        [0.8913, 0.2080, 0.8834],
        [0.3954, 0.4594, 0.3046],
        [0.4344, 0.8164, 0.1421],
        [0.8918, 0.9171, 0.4752]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [3]:
z = x + y
print(z)

tensor([[1.7739, 1.9195, 1.8252],
        [1.8913, 1.2080, 1.8834],
        [1.3954, 1.4594, 1.3046],
        [1.4344, 1.8164, 1.1421],
        [1.8918, 1.9171, 1.4752]])


# Advantages of Pytorch over Numpy
## Automatic Differentiation
## Running on GPUs

In [4]:
df = pd.read_csv('tictactoemoves.csv')

In [5]:
print(df.head())

   C1  C2  C3  C4  C5  C6  C7  C8  C9  player  move
0   1   1   1   1   0   0   1   0   0      -1    -1
1   1   1   1   1   0   0   0   1   0      -1    -1
2   1   1   1   1   0   0   0   0   1      -1    -1
3   1   1   1   1   0   0   0  -1  -1      -1    -1
4   1   1   1   1   0   0  -1   0  -1      -1    -1


In [6]:
X = df[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'player']].values
y = df['move'].values

In [7]:
print(y)

[-1 -1 -1 ...  3  5  1]


In [8]:
y_encoded = np.zeros([len(y), 10])

for i in range(0, len(y)):
    if(y[i] == -1):
        y_encoded[i][0] = 1.0
    else:
        y_encoded[i][int(y[i])] = 1.0

In [9]:
print(y_encoded[-2])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [10]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.float32)

In [11]:
print(y_tensor)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])


### train,test sets

In [13]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.1, random_state=42)

In [18]:
print(len(X_test))

548


## NN Model

In [19]:
import torch.nn as nn

In [20]:
class TicTacToeTrainer(nn.Module):

    def __init__(self):
        super(TicTacToeTrainer, self).__init__()

        self.fc1 = nn.Linear(10, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10) #logits as output

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)

        return x
       

In [21]:
model = TicTacToeTrainer()

In [22]:
x_eval = torch.tensor([[0,-1,-1,-1,-1,0,1,1,-1,1]],dtype=torch.float32)
output = model(x_eval)


In [23]:
print(output)

tensor([[ 0.1437,  0.0089, -0.1534,  0.0569,  0.0435, -0.0368, -0.0916,  0.0232,
          0.1770,  0.0488]], grad_fn=<AddmmBackward0>)


# Training

## Softmax + Cross-Entropy

In [24]:
criterion = nn.CrossEntropyLoss()

## Optimizer

In [25]:
import torch.optim as optim

In [26]:
optimizer =  optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

## Training

In [27]:
X_test = X_tensor
X_train = X_tensor

y_test = y_tensor
y_train = y_tensor

In [28]:
num_epochs = 1000 # 1 epoch, 1 traversal through the training set
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad() #zero out the gradients
    outputs = model(X_train) #run the forward pass
    loss = criterion(outputs, y_train) # compute the loss
    loss.backward() #gradient calculation happens from here
    optimizer.step() #takes the step based on the gradients and lr

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.6f}')

Epoch [10/1000], Loss: 2.277314
Epoch [20/1000], Loss: 2.209173
Epoch [30/1000], Loss: 2.109079
Epoch [40/1000], Loss: 2.009355
Epoch [50/1000], Loss: 1.903062
Epoch [60/1000], Loss: 1.778257
Epoch [70/1000], Loss: 1.640190
Epoch [80/1000], Loss: 1.509634
Epoch [90/1000], Loss: 1.394889
Epoch [100/1000], Loss: 1.291169
Epoch [110/1000], Loss: 1.201662
Epoch [120/1000], Loss: 1.128224
Epoch [130/1000], Loss: 1.067955
Epoch [140/1000], Loss: 1.019023
Epoch [150/1000], Loss: 0.980257
Epoch [160/1000], Loss: 0.946484
Epoch [170/1000], Loss: 0.915157
Epoch [180/1000], Loss: 0.886362
Epoch [190/1000], Loss: 0.860082
Epoch [200/1000], Loss: 0.836583
Epoch [210/1000], Loss: 0.817214
Epoch [220/1000], Loss: 0.801255
Epoch [230/1000], Loss: 0.787196
Epoch [240/1000], Loss: 0.774373
Epoch [250/1000], Loss: 0.762388
Epoch [260/1000], Loss: 0.750729
Epoch [270/1000], Loss: 0.739008
Epoch [280/1000], Loss: 0.727578
Epoch [290/1000], Loss: 0.716667
Epoch [300/1000], Loss: 0.706459
Epoch [310/1000], L

In [29]:
# Evaluating
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    _, actual = torch.max(y_test.data, 1)

    accuracy = (predicted == actual).sum().item()/actual.size(0)*100.0

    print(f'Test Accuracy : {accuracy}')

Test Accuracy : 83.78970427163198


In [30]:
# Evaluating
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    _, predicted = torch.max(outputs.data, 1)
    _, actual = torch.max(y_train.data, 1)

    accuracy = (predicted == actual).sum().item()/actual.size(0)*100.0

    print(f'Training Accuracy : {accuracy}')

Training Accuracy : 83.78970427163198


In [31]:
invalid = 0
invalid_states = []
for i in range(0, len(predicted)):
    isEmpty = False 
    if(actual[i] != predicted[i]):
        isEmpty = False 
        if(X_test[i,predicted[i]-1] == -1):
            isEmpty = True
            
        else:
            invalid = invalid + 1
            invalid_states.append({X_test[i].to(torch.int32),actual[i],predicted[i]})
        print(X_test[i].to(torch.int32),actual[i],predicted[i], isEmpty)


tensor([ 1,  1,  0,  1,  0, -1, -1,  0,  1,  0], dtype=torch.int32) tensor(7) tensor(6) True
tensor([ 1,  1,  0,  1, -1,  0, -1,  0, -1,  1], dtype=torch.int32) tensor(7) tensor(5) True
tensor([ 1,  1,  0,  1, -1, -1, -1,  0, -1,  0], dtype=torch.int32) tensor(7) tensor(5) True
tensor([ 1,  1,  0,  1, -1, -1, -1, -1,  0,  0], dtype=torch.int32) tensor(6) tensor(5) True
tensor([ 1,  1,  0,  0,  1,  1,  0, -1, -1,  0], dtype=torch.int32) tensor(8) tensor(9) True
tensor([ 1,  1,  0,  0,  1,  0, -1, -1, -1,  1], dtype=torch.int32) tensor(8) tensor(9) True
tensor([ 1,  1,  0,  0,  1, -1,  0, -1, -1,  1], dtype=torch.int32) tensor(6) tensor(9) True
tensor([ 1,  1,  0,  0, -1, -1,  1, -1,  0,  1], dtype=torch.int32) tensor(6) tensor(5) True
tensor([ 1,  1,  0,  0, -1, -1,  1, -1, -1,  0], dtype=torch.int32) tensor(6) tensor(5) True
tensor([ 1,  1,  0,  0, -1, -1, -1, -1, -1,  1], dtype=torch.int32) tensor(5) tensor(7) True
tensor([ 1,  1,  0, -1,  1,  1, -1,  0,  0,  0], dtype=torch.int32) te

In [32]:
torch.save(model.state_dict(), "TicTacToe_OverfittedWeights.pth")